In [ ]:
import torch
from torch import nn
from d2l import torch as d2l
class Reshape(torch.nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)
net=torch.nn.Sequential(Reshape(),nn.Vonv2d(1,6,kernel_size=5,padding=2,nn.Sigmoid(),nn.AvgPool2d(kernel_size=2, stride=2),
))

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils import data
import torch.optim as optim

# prepare dataset
 
batch_size = 64
def load_data_fashion_mnist(batch_size, resize=None): 
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307, ), (0.3081, ))])
    mnist_train = datasets.FashionMNIST(
        root="../data", train=True, transform=transform, download=True)
    mnist_test = datasets.FashionMNIST(
        root="../data", train=False, transform=transform, download=True)
    return data.DataLoader(mnist_train, batch_size, shuffle=True),data.DataLoader(mnist_test, batch_size, shuffle=False)
# design model using class
 
train_loader,test_loader= load_data_fashion_mnist(batch_size, resize=28)
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
 
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
 
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
 
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
 
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
 
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
 
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
 
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
 
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim=1) # b,c,w,h  c对应的是dim=1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5) # 88 = 24x3 + 16
 
        self.incep1 = InceptionA(in_channels=10) # 与conv1 中的10对应
        self.incep2 = InceptionA(in_channels=20) # 与conv2 中的20对应
 
        self.mp = nn.AvgPool2d(2)
        self.fc = nn.Linear(1408, 10) 
 
 
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
 
        return x
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

D:\csrgzn\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


[1,   300] loss: 1.159
[1,   600] loss: 0.726
[1,   900] loss: 0.653
accuracy on test set: 75 % 
[2,   300] loss: 0.596
[2,   600] loss: 0.569
[2,   900] loss: 0.535
accuracy on test set: 79 % 
[3,   300] loss: 0.505
[3,   600] loss: 0.492
[3,   900] loss: 0.487
accuracy on test set: 77 % 
[4,   300] loss: 0.462
[4,   600] loss: 0.457
[4,   900] loss: 0.437
accuracy on test set: 83 % 
[5,   300] loss: 0.420
[5,   600] loss: 0.431
[5,   900] loss: 0.417
accuracy on test set: 83 % 
[6,   300] loss: 0.409
[6,   600] loss: 0.396
[6,   900] loss: 0.398
accuracy on test set: 85 % 
[7,   300] loss: 0.393
[7,   600] loss: 0.386
[7,   900] loss: 0.374
accuracy on test set: 85 % 
[8,   300] loss: 0.372
[8,   600] loss: 0.369
[8,   900] loss: 0.369
accuracy on test set: 86 % 
[9,   300] loss: 0.351
[9,   600] loss: 0.368
[9,   900] loss: 0.352
accuracy on test set: 86 % 
[10,   300] loss: 0.346
[10,   600] loss: 0.345
[10,   900] loss: 0.343
accuracy on test set: 86 % 
